In [ ]:
import cv2
import os
import torch
import gdown
import random
import easydict 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import torchvision.datasets as dset
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score
from utils.deepsvdd import DeepSVDD_network, pretrain_autoencoder, TrainerDeepSVDD

In [ ]:
gdown.download('https://drive.google.com/uc?id=15h8EUZOb4qJHy5zE9JDCSCNW28I1xKKG','./images/attack_models.zip',quiet=False)

In [ ]:
os.system("unzip ./images/attack_models.zip -d ./images/")

In [ ]:
gdown.download('https://drive.google.com/uc?id=1pC3d0nrqB6OtjjN8sKtwI0ebV-RLzIAG','./models/attack_models.zip',quiet=False)

In [ ]:
os.system("unzip ./models/attack_models.zip -d ./models/")

In [ ]:
# IMPORTANT
# PLEASE CHANGE THE VAULUE OF DeepTaster_forder TO THE DEEPTASTER DIRECTORY 
Home_directory = ''
DeepTaster_forder = Home_directory+"/DeepTaster/"

In [ ]:
# Specify the cuda setting if you needed
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
"""
Detection Classifier Generation
"""

import os
import numpy as np
import torch
import easydict 
import random
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torchvision import transforms
from utils.deepsvdd import DeepSVDD_network, pretrain_autoencoder, TrainerDeepSVDD

In [ ]:
# load train adversarialDFT dataset
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/train',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

train_dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=16,
                                        shuffle=True,
                                        num_workers=8)

In [ ]:
# load test adversarialDFT dataset
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/val',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

val_dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=16,
                                        shuffle=False,
                                        num_workers=8)

In [ ]:
# Specify the cuda setting if you needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args = easydict.EasyDict({
        'num_epochs':3,
        'num_epochs_ae':40,
        'lr':1e-3,
        'lr_ae':1e-3,
        'weight_decay':5e-7,
        'weight_decay_ae':5e-3,
        'lr_milestones':[50],
        'batch_size':1024,
        'pretrain':True,
        'latent_dim':32,
        'normal_class':0
                })

In [ ]:
deep_SVDD = TrainerDeepSVDD(args, train_dataloader,device, DeepTaster_forder +'models/classifier/pretrained.pth')
if args.pretrain:
    deep_SVDD.pretrain()
net = DeepSVDD_network().to(device)
state_dict = torch.load(DeepTaster_forder +'models/classifier/pretrained.pth')
net.load_state_dict(state_dict['net_dict'])
c = torch.Tensor(state_dict['center']).to(device)

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=args.lr,
                        weight_decay=args.weight_decay)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
            milestones=args.lr_milestones, gamma=0.1)


In [ ]:
net.train()
for epoch in range(args.num_epochs):
    total_loss = 0
    for x, _ in train_dataloader:
        x = x.float().to(device)

        optimizer.zero_grad()
        z = net(x)
        loss = torch.mean(torch.sum((z - c) ** 2, dim=1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    scheduler.step()
    print('Training Deep SVDD... Epoch: {}, Loss: {:.3f}'.format(
            epoch+1, total_loss/len(train_dataloader)))

In [ ]:
scores = []
label = []
net.eval()
with torch.no_grad():
    for x, y in val_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1).cpu()
        for i in range(16):
            if y[i]==0:
                scores.append(score[i])
                label.append(0)
            else:
                scores.append(score[i])
                label.append(1)     

In [ ]:
scores.sort()

In [ ]:
threshold = float(scores[862])

In [ ]:
print("threshold: ", threshold)
threshold_file=open(DeepTaster_forder +'models/classifier/threshold.txt','w')
threshold_file.write(str(threshold))
threshold_file.close()
torch.save(net, DeepTaster_forder +'models/classifier/deepsvdd.pt')

In [ ]:
net = DeepSVDD_network().to(device)
state_dict = torch.load(DeepTaster_forder +'models/classifier/pretrained.pth')
c = torch.Tensor(state_dict['center']).to(device)
net=torch.load(DeepTaster_forder +'models/classifier/deepsvdd.pt').to(device)
threshold_file=open(DeepTaster_forder +'models/classifier/threshold.txt','r')
threshold=float(threshold_file.readline())
threshold_file.close()

In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/test',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
detection_rate =[]
architecture_list = ["DenseNet", "ResNet","VGG"]
dataset = ["CIFAR10", "MNIST","Tiny-ImageNet","ImageNet"]
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            if y[0]==0:
                detection_rate.append(100 if attack_probability >=144 else 0)
            else:
                detection_rate.append(100 if attack_probability < 144 else 0)
            temp=0
            attack_probability=0

print("MAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Victim","Suspect", "Ground Truth","Architecture","Attack Prob.","Copy Detection (%)"))
for i in range(3):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("CIFAR10", dataset[int(i/3)], "Stolen", architecture_list[i%3],round(result_list[i]/288,2), detection_rate[i] ))
for i in range(3,12):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("CIFAR10", dataset[int(i/3)], "Benign", architecture_list[i%3],round(result_list[i]/288,2), detection_rate[i] ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/DAA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print(dataset.class_to_idx)

In [ ]:
print("DAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["100","20","40","60","80"]
for i in [1,2,3,4,0]:
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/DAA/DAA_'+epochs[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", epochs[i],round(acc,2), round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/SAA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("SAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Victim","Suspect", "Ground Truth","Used Dataset","Attack Prob.","Copy Detection (%)"))
data_percentage = ["10", "30","50","70","90"]
for i in range(5):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10','CIFAR10', "Stolen", data_percentage[i],round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/TLA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("TLA results\n")
print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = ["2","4","6","8"]
for i in range(4):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/TLA/Transfer_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10 to MNIST', "Stolen", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/MFA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("MFA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Used Dataset","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = [ "500","1000","2500"]
for i in range(3):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/MFA/Finetuning_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))
for i in range(3,6):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/MFA/MNIST_Finetuning_'+data_percentage[i-4]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST', "Benign", data_percentage[i-3],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] <144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/MPA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("MPA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Pruned (%)","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = ["20", "40","60"]
for i in range(3,6):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/MPA/Prune_'+data_percentage[i-3]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", data_percentage[i-3],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i]  >=144 else 0 ))
for i in range(3):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/MPA/MNIST_Prune_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST', "Benign", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] <144 else 0 ))


In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/DATLA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print(dataset.class_to_idx)

In [ ]:
print("DATLA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["100","20","40","60","80"]
for i in [1,2,3,4,0]:
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/DATLA/DATLA_'+epochs[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", epochs[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))

In [ ]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/TLPA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [ ]:
print("TLPA results\n")
print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Used Dataset","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["10", "30","50","70",'90']
for i in range(5):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/TLPA/TLPA_'+epochs[i][:1]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST to CIFAR10', "Stolen", epochs [i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))